In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
data_train = pd.read_csv('final data/features.csv', index_col='match_id')
data_train.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [3]:
data_train0 = data_train.copy()

In [4]:
data_train[['r1_hero', 'first_blood_player1']]

,r1_hero,first_blood_player1
match_id,,
0,11,9.0
1,42,7.0
2,33,3.0
3,29,NaN
4,13,6.0
...,...,...
114402,47,2.0
114403,43,9.0
114404,98,3.0


In [5]:
data_train.isnull().sum()[data_train.isnull().sum()>0]

first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
dtype: int64

In [34]:
data_train['radiant_f_b'] = [1 if x==1 else 0 for x in data_train.first_blood_team]
data_train['dire_f_b'] = [1 if x==0 else 0 for x in data_train.first_blood_team]

In [35]:
from sklearn.decomposition import PCA
features_pca = ['radiant_bottle_time', 'radiant_courier_time','radiant_flying_courier_time', 
     'radiant_tpscroll_count', 'radiant_boots_count','radiant_ward_observer_count',
     'radiant_ward_sentry_count','radiant_first_ward_time', 'dire_bottle_time', 'dire_courier_time',
     'dire_flying_courier_time', 'dire_tpscroll_count', 'dire_boots_count', 'dire_ward_observer_count',
     'dire_ward_sentry_count', 'dire_first_ward_time']

data_for_pca = data_train[features_pca].copy()
data_for_pca.fillna(0, inplace=True)    
pca = PCA(n_components=4)
data_for_pca = pca.fit_transform(data_for_pca)

In [36]:
#data_for_pca = pd.DataFrame(data_for_pca, columns=[['pca1', 'pca2', 'pca3', 'pca4']])
#data_train = pd.concat([data_train, data_for_pca])
tmps = ['pca1', 'pca2', 'pca3', 'pca4']
for i, tmp in enumerate(tmps):
    data_train[tmp] = data_for_pca[:,i]

In [37]:
data_train = data_train.drop(columns=features_pca)

In [17]:
data_train.isnull().sum()[data_train.isnull().sum()>0]

first_blood_time       19553
first_blood_player1    19553
first_blood_player2    43987
dtype: int64

In [18]:
print(*data_train.columns)

lobby_type r1_hero r1_level r1_xp r1_gold r1_lh r1_kills r1_deaths r1_items r2_hero r2_level r2_xp r2_gold r2_lh r2_kills r2_deaths r2_items r3_hero r3_level r3_xp r3_gold r3_lh r3_kills r3_deaths r3_items r4_hero r4_level r4_xp r4_gold r4_lh r4_kills r4_deaths r4_items r5_hero r5_level r5_xp r5_gold r5_lh r5_kills r5_deaths r5_items d1_hero d1_level d1_xp d1_gold d1_lh d1_kills d1_deaths d1_items d2_hero d2_level d2_xp d2_gold d2_lh d2_kills d2_deaths d2_items d3_hero d3_level d3_xp d3_gold d3_lh d3_kills d3_deaths d3_items d4_hero d4_level d4_xp d4_gold d4_lh d4_kills d4_deaths d4_items d5_hero d5_level d5_xp d5_gold d5_lh d5_kills d5_deaths d5_items first_blood_time first_blood_player1 first_blood_player2 duration radiant_win tower_status_radiant tower_status_dire barracks_status_radiant barracks_status_dire radiant_f_b dire_f_b pca1 pca2 pca3 pca4


In [38]:
X = data_train.drop(columns=['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire',
                           'barracks_status_radiant', 'barracks_status_dire'])
y = data_train.radiant_win

In [39]:
X1 = X.copy()
X.fillna(10000, inplace=True)
X1.fillna(0, inplace=True)

In [40]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

In [22]:
divider = KFold(5, shuffle=True, random_state=11)

In [23]:
import time
import datetime

In [41]:
start_time = datetime.datetime.now()

model = GradientBoostingClassifier()
score = sum(cross_val_score(model, X1.values, y.values, cv=divider.split(X1, y), scoring='roc_auc', n_jobs = -1))/5

dt1 = datetime.datetime.now() - start_time

print('Time elapsed:', dt1)
print('score:', score)

Time elapsed: 0:07:51.396436
score: 0.7056203093655669


In [27]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
scaler = StandardScaler()

In [28]:
X1_scaled = scaler.fit_transform(X1)
X_scaled = scaler.fit_transform(X)

In [29]:
gb = GradientBoostingClassifier()
score = sum(cross_val_score(gb, X1_scaled, y, cv=divider.split(X1_scaled, y), scoring='roc_auc', n_jobs = -1))/5
print(score)

0.7054372685521848


In [43]:
gs_params = {'C':[10**n for n in range(-4, 4)]}
lr = LogisticRegression()

gs = GridSearchCV(lr, gs_params, scoring='roc_auc', cv=divider, n_jobs=-1) # n_jobs=-1 для распараллеливания
gs.fit(X1_scaled, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=11, shuffle=True),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [44]:
c = gs.best_params_
print(c)

{'C': 0.01}


In [45]:
start_time = datetime.datetime.now()


best_lr = LogisticRegression(C=0.01)
score = sum(cross_val_score(best_lr, X_scaled, y, cv=divider.split(X_scaled,y), scoring='roc_auc', n_jobs = -1))/5

dt2 = datetime.datetime.now() - start_time

print('Time elapsed:', dt2)
print('for scaled X score:', score)

score = sum(cross_val_score(best_lr, X1_scaled, y, cv=divider.split(X1_scaled,y), scoring='roc_auc', n_jobs = -1))/5
print('for scaled X1 score:', score)

Time elapsed: 0:00:03.964484
for scaled X score: 0.7155802355791934
for scaled X1 score: 0.7154869876875882


при удалении столбцов features_pca и first blood

for scaled X score: 0.7164300157019141

for scaled X1 score: 0.7163598157814205

градиентный бустинг примерно одинаков для любых данных (масштабированых или нет). Новые признаки существенного прироста качества не дают на обоих моделях.